In [4]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
from pixelmed_calc.medical_imaging.modelstore import get_model_from_name
from pixelmed_calc.medical_imaging.DLtools.process import load_data_from_csv,get_file_and_labels,get_transforms,train_and_validate
from monai.data import ImageDataset
import pandas as pd
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.nn.functional as F

# CSV文件路径
csv_path = r'E:\127lungnode\lungnode\group.csv'
train_base_path = r'E:\127lungnode\lungnode\crop3d'
val_base_path = r'E:\127lungnode\lungnode\crop3d'

#tfs=get_transforms()
tfs=get_transforms(resize=[16,128,128])
# 读取所有数据
file_names, labels, groups = load_data_from_csv(csv_path)

file_train, train_labels = get_file_and_labels(file_names, labels, groups, 'train', train_base_path)

file_val, val_labels = get_file_and_labels(file_names, labels, groups, 'val', val_base_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#model = get_model_from_name['resnet50_3D'](num_classes=2, pretrained=True, in_channels=1).to(device)
model=get_model_from_name['vit_3D'](
    image_size = 128,          # image size
    frames = 16,               # number of frames
    image_patch_size = 16,     # image patch size
    frame_patch_size = 2,      # frame patch size
    num_classes = 2,
    channels=1,
    dim = 1024,
    depth = 6,
    heads = 8,
    mlp_dim = 2048,
    dropout = 0.1,
    emb_dropout = 0.1
).to("cuda")
model.load_state_dict(torch.load(r"D:\pixelmedAI\function_pm\note2-深度学习分类\results\best_model.pth"))

<class 'monai.transforms.utility.array.AddChannel'>: Class `AddChannel` has been deprecated since version 0.8. please use MetaTensor data type and monai.transforms.EnsureChannelFirst instead.


<All keys matched successfully>

In [5]:
from pixelmed_calc.medical_imaging.DLtools.utils import save_prob_csv,eval_model
val_ds = ImageDataset(image_files=file_val, labels=val_labels, transform=tfs)
val_loader = DataLoader(val_ds, batch_size=1, shuffle=False, num_workers=0, pin_memory=torch.cuda.is_available())

val_metrics = eval_model(model, val_loader, device)  
save_prob_csv(val_metrics, os.path.join('results', f'test1.csv'))  